In [2]:
import nltk
import pandas
import numpy as np
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.classify import MaxentClassifier
from nltk import classify
import re

### 1. Importamos el corpus de comentarios

In [3]:
comentarios = pandas.read_csv("comentarios_peliculas.csv", skiprows=1, delimiter=';', skip_blank_lines=True, encoding='utf-8')

### 1.1 Realizamos un breve analisis del corpus 

In [4]:

positivos = 0 
neutros = 0
negativos = 0
total = str(len(comentarios))
largo_comentario = 0
lista_comentarios = comentarios.values.tolist()
for x in lista_comentarios:
    if x[7] < 3:
        negativos += 1
    elif x[7] == 3:
        neutros += 1
    else:
        positivos += 1
    largo_comentario += len(re.findall(r'\w+', x[6]))


avg_positivos = str(float(positivos)*100 / float(total))
avg_negativos = str(float(negativos)*100 / float(total))
avg_neutros = str(float(neutros)*100 / float(total))
avg_largo_comentario = str(float(largo_comentario)/ float(total))
        
print ("El corpus tiene " + total + " comentarios")
print ("Hay " + str(positivos) + " comentarios positivos lo que equivale al  " + avg_positivos[:4] + "%")
print ("Hay " + str(neutros) + " comentarios neutros lo que equivale al  " + avg_neutros[:4] + "%")
print ("Hay " + str(negativos) + " comentarios negativos lo que equivale al  " + avg_negativos[:4] + "%")
print ("El largo de palabras promedio de los comentarios es " + avg_largo_comentario[:2])

El corpus tiene 1447 comentarios
Hay 859 comentarios positivos lo que equivale al  59.3%
Hay 222 comentarios neutros lo que equivale al  15.3%
Hay 366 comentarios negativos lo que equivale al  25.2%
El largo de palabras promedio de los comentarios es 38


### 2. Separamos entrenamiento y testeo, lo pasamos a listas comunes y eliminamos datos que no utilizamos. 

In [5]:
np.random.seed(42) 
msk = np.random.rand(len(comentarios)) < 0.8
comentarios_training= comentarios[msk] 
comentarios_testing = comentarios[~msk] 

lista_comentarios_training = comentarios_training.values.tolist()
for x in lista_comentarios_training:
    del x[8]
    del x[5]
    del x[4]
    del x[3]
    del x[2]
    del x[1]
    del x[0]
print ("El corpus de entrenamiento tiene largo " + str(len(lista_comentarios_training)))
lista_comentarios_testing = comentarios_testing.values.tolist()
for x in lista_comentarios_testing:
    del x[8]
    del x[5]
    del x[4]
    del x[3]
    del x[2]
    del x[1]
    del x[0]
print ("El corpus de testing tiene largo " + str(len(lista_comentarios_testing)))

El corpus de entrenamiento tiene largo 1147
El corpus de testing tiene largo 300


### 3. Obtenemos las palabras mas comunes del corpus, quitando puntuación y stopwords del Español

In [6]:
def word_feats(commentWords):
    commentWords = tokenizer.tokenize(commentWords)
    return dict([(word[0], word[0] in commentWords) for word in palabras_features])

def classification(value):
    if value > 3:
        classification = "negative"
    elif value == 3:
        classification = "neutro"
    else:
        classification = "positive"
    return classification

textoComentariosTraining = ""
for x in lista_comentarios_training:
    textoComentariosTraining += " " + x[0].lower()

tokenizer = RegexpTokenizer(r'\w+')
corpus_text = tokenizer.tokenize(textoComentariosTraining) 
stopwordsEspanol = nltk.corpus.stopwords.words('spanish')
stopwordsEspanol.remove("no")
stopwordsEspanol.remove("sí")
palabrasMasComunes = FreqDist(w.lower() for w in corpus_text if w not in stopwordsEspanol)
listaValores = [10,20,30,40,50,60,70,80,90,100]
listaAccuracy = []
for i in listaValores:
    print ("Para n =",i)
    palabras_features = palabrasMasComunes.most_common(i)
    array_for_training = [(word_feats(f[0]), classification(f[1])) for f in lista_comentarios_training]
    classifier = MaxentClassifier.train(array_for_training, max_iter=10)
    array_for_test = [(word_feats(f[0]), classification(f[1])) for f in lista_comentarios_testing]
    accuracy = classify.accuracy(classifier,array_for_test)
    listaAccuracy.append(accuracy)
    
print(listaAccuracy)
    

Para n = 10
  ==> Training (10 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.251
             2          -0.79943        0.597
             3          -0.78879        0.597
             4          -0.77962        0.597
             5          -0.77172        0.597
             6          -0.76490        0.597
             7          -0.75899        0.598
             8          -0.75385        0.594
             9          -0.74935        0.595
         Final          -0.74541        0.597
Para n = 20
  ==> Training (10 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.251
             2          -0.80278        0.597
             3          -0.79387        0.597
             4          -0.78565        0.597
             5          -0.77810        0.597
             6          -0.77115      